# Pre and Post Fed Inflation Study 

## Imports 

In [ ]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import pingouin as pg

In [ ]:
PROJECT_DIRECTORY = "./"
CSV_DATA_DIRECTORY = os.path.join(PROJECT_DIRECTORY, "data_as_csv")
INFLATION_DATA_CSV = "inflation.csv"

In [ ]:
YEAR_COLUMN = "Year"
ANNUAL_PERCENT_CHANGE_COLUMN = "Annual Percent Change"

# will create a synthetic column "Inflation"
INFLATION_COLUMN = "Inflation"

# alpha corresponding to 95% confidence in our statistical tests 
ALPHA = 0.05

## Load Raw Data

In [ ]:
# source https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1800-

df_inflation_org= pd.read_csv(Path(CSV_DATA_DIRECTORY) / INFLATION_DATA_CSV)

df_inflation_org.columns

In [ ]:
df_inflation_org.dtypes

In [ ]:
df_inflation = df_inflation_org.copy()
conversions = { ANNUAL_PERCENT_CHANGE_COLUMN: "string"}
df_inflation = df_inflation.astype(conversions).copy()
df_inflation.convert_dtypes()

df_inflation[INFLATION_COLUMN] = df_inflation[ANNUAL_PERCENT_CHANGE_COLUMN].apply(lambda s: float(s.strip()[:-1])/100)
df_inflation

In [ ]:
df_inflation.dtypes

In [ ]:
sns.lineplot(data=df_inflation, x=YEAR_COLUMN, y=INFLATION_COLUMN)

df_inflation_pre_fed_1872 = df_inflation[(df_inflation[YEAR_COLUMN] <= 1913) & (df_inflation[YEAR_COLUMN] >= 1872)].copy()
df_inflation_pre_fed_1801 = df_inflation[df_inflation[YEAR_COLUMN] <= 1913].copy()
df_inflation_post_fed = df_inflation[df_inflation[YEAR_COLUMN] > 1913].copy()

periods = {"pre_fed_1801": df_inflation_pre_fed_1801, "pre_fed_1872": df_inflation_pre_fed_1872, "post_fed": df_inflation_post_fed}

for period_name, period_data in periods.items():
    n = period_data[INFLATION_COLUMN].count()
    mean = period_data[INFLATION_COLUMN].mean()
    std = period_data[INFLATION_COLUMN].std() # should actually be a population std
    print(f"{period_name} n: {n} mean: {mean} std: {std}")

In [ ]:
# do a t-tests (which will be z-tests since n > 30 for all samples

post_fed_inflation_data_points = np.array(df_inflation_post_fed[INFLATION_COLUMN]) 

def inflation_z_test(df_pre_fed: pd.DataFrame) -> pd.DataFrame:
    pre_fed_data_data_points = np.array(df_pre_fed[INFLATION_COLUMN])
    return pg.ttest(pre_fed_data_data_points, post_fed_inflation_data_points, alternative="less")

inflation_z_test(df_inflation_pre_fed_1872)

In [ ]:
inflation_z_test(df_inflation_pre_fed_1801) 

# Conclusion:

Descriptive statistics suggests that inflation was lower pre-Fed:

- pre_fed_1801 (n=113) mean: -0.0032831858407079647 std: 0.05643793509981763
- pre_fed_1872 (n=42) mean: -0.004190476190476191 std: 0.02688842174910896
- post_fed (n=110) mean: 0.03268181818181819 std: 0.04709699766783905

Note, however, that the standard deviation is higher pre-Fed if years from 1801 to 1913 are considered but lower if only 1872 to 1913 are considered.



Applying one-sided two sample z-tests shows that if 1872 to 1903 is considered, we can say, with 95% confidence, that inflation was at least 3% lower, on average, pre-Fed. 

If 1801 to 1903 is considered, then we can say, with 95% confidence that inflation was at least 2% lower, on average, pre-Fed.


The obvious caveat is that correlation is, of course, not causation. This does not prove that there is more inflation when there is a central bank.

More speculatively, the lower inflation could be due to harsher depressions/deflationary periods pre-Fed. 